## bin to xyz 

In [2]:
import struct
import numpy as np
file_path = 'D:\VS2022Projects/tvm-editing-master\TVMEditor.Test\Data\pent\centers\mesh_000res_1000_000.bin'

print(file_path)

def load_bin(file_path):
    with open(file_path, 'rb') as f:
        # Read the first 4 bytes as an integer (this is the count 'c')
        c = struct.unpack('i', f.read(4))[0]
        
        # Initialize an empty numpy array to store the vectors
        vectors = np.zeros((c, 3), dtype=np.float32)
        
        for i in range(c):
            # Read the next 12 bytes (3 floats) for the Vector3
            x, y, z = struct.unpack('3f', f.read(12))
            vectors[i] = (x, y, z)
    
    return vectors

centers = load_bin(file_path)
print(centers, centers.__len__())

D:\VS2022Projects/tvm-editing-master\TVMEditor.Test\Data\pent\centers\mesh_000res_1000_000.bin
[[ 0.30941448  0.9935851  -0.10177808]
 [-0.01174076  0.28923604  0.2646991 ]
 [-0.3199594   0.37756237 -0.21296251]
 ...
 [ 0.03638093  1.5659114  -0.29961684]
 [ 0.00987653  0.13647208  0.1315317 ]
 [ 0.33245873  0.64176136 -0.19910005]] 1000


In [4]:
file_path = 'D:\VS2022Projects/tvm-editing-master\TVMEditor.Test\Data\pent\centers\mesh_000res_1000_000_out.bin'
centers = load_bin(file_path)
print(centers, centers.__len__())

[[ 0.30941448  0.9935851  -0.10177808]
 [-0.01174076  0.28923604  0.2646991 ]
 [-0.3199594   0.37756237 -0.21296251]
 ...
 [ 0.03638093  1.5659114  -0.29961684]
 [ 0.00987653  0.13647208  0.1315317 ]
 [ 0.33245873  0.64176136 -0.19910005]] 1000


In [42]:
print(centers[4])

[ 0.20597723  1.8731184  -0.02225684]


## xyz to bin

In [2]:
import struct
import numpy as np

def save_bin(file_path, centers):
    # Open the file in write-binary mode
    with open(file_path, 'wb') as f:
        # Write the number of vectors as a 4-byte integer
        c = len(centers)
        f.write(struct.pack('i', c))
        
        # Write each vector (x, y, z) as three 4-byte floats
        for vector in centers:
            f.write(struct.pack('3f', *vector))

# Example usage
new_file_path = 'D:\VS2022Projects/tvm-editing-master\TVMEditor.Test\Data\pent\centers\mesh_000res_1000_000_out.bin'

# Save the centers array back to a bin file
save_bin(new_file_path, centers)

## draw centers

In [3]:
import open3d as o3d
import numpy as np
spheres = []

# Define the radius of the spheres
sphere_radius = 0.05  # Adjust the radius as needed

# Create a mesh sphere for each point and translate it to the point's location
for point in centers:
    # Create a sphere mesh
    mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=sphere_radius)
    mesh_sphere.compute_vertex_normals()
    
    # Translate the sphere to the point's location
    mesh_sphere.translate(point)
    
    # Add the sphere to the list
    spheres.append(mesh_sphere)

# Combine all the spheres into one mesh
all_spheres = o3d.geometry.TriangleMesh()
for sphere in spheres:
    all_spheres += sphere

# Optionally, you can set the color of the spheres
all_spheres.paint_uniform_color([0.7, 0.7, 0.7])  # Light grey color

# Draw the combined mesh
o3d.visualization.draw_geometries([all_spheres])

## Apply Transformation

In [4]:
file_path = 'D:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\pent/transformations_1.txt'

# Initialize a list to store the parsed data
real_data = []
dual_data = []

# Open the file and read each line
with open(file_path, 'r') as file:
    for line in file:
        # Strip any leading/trailing whitespace and split the line by semicolon
        values = line.strip().split(';')
        
        # Convert the string values to floats
        real_values = tuple(float(value) for value in values[:4])  # First four as Real
        dual_values = tuple(float(value) for value in values[4:])  # Last four as Dual
        
        # Append the Real and Dual parts to their respective lists
        real_data.append(real_values)
        dual_data.append(dual_values)

def create_matrix_3x3(real):
    # Unpack the real and dual components
    rx, ry, rz, rw = real
    
    # Initialize the matrix
    matrix = np.zeros((3, 3))
    
    # Rotation part of the matrix
    matrix[0, 0] = rw * rw + rx * rx - ry * ry - rz * rz
    matrix[0, 1] = 2 * rx * ry - 2 * rw * rz
    matrix[0, 2] = 2 * rx * rz + 2 * rw * ry
    matrix[1, 0] = 2 * rx * ry + 2 * rw * rz
    matrix[1, 1] = rw * rw - rx * rx + ry * ry - rz * rz
    matrix[1, 2] = 2 * ry * rz - 2 * rw * rx
    matrix[2, 0] = 2 * rx * rz - 2 * rw * ry
    matrix[2, 1] = 2 * ry * rz + 2 * rw * rx
    matrix[2, 2] = rw * rw - rx * rx - ry * ry + rz * rz
    
    
    return matrix

print(real_data.__len__())

1000


In [33]:
def quaternion_multiply(quaternion1, quaternion0):
    x0, y0, z0, w0 = quaternion0
    x1, y1, z1, w1 = quaternion1
    return np.array([-x1*x0 - y1*y0 - z1*z0 + w1*w0,
                         x1*w0 + y1*z0 - z1*y0 + w1*x0,
                        -x1*z0 + y1*w0 + z1*x0 + w1*y0,
                         x1*y0 - y1*x0 + z1*w0 + w1*z0], dtype=np.float64)

In [73]:
from scipy.spatial.transform import Rotation as R
transformed_centers = []
Transformations = []
for i in range(len(centers)):
    real_values = real_data[i]  # Replace with actual Real values
    dual_values = dual_data[i]  # Replace with actual Dual values
    r = R.from_quat(real_data[i])
    Transformations.append(r.as_matrix())
    point = centers[i]
    x1 = real_values[0]
    y1 = real_values[1]
    z1 = real_values[2]
    w1 = real_values[3]
    x2 = dual_values[0]
    y2 = dual_values[1]
    z2 = dual_values[2]
    w2 = dual_values[3]
    Conjugate = (-x1,-y1,-z1, w1)
    TQ = quaternion_multiply(dual_values, Conjugate)
    Vector = (TQ[1]*2, TQ[2]*2, TQ[3]*2) # not TQ[0]*2, TQ[1]*2, TQ[2]*2, TQ[0] is w 
    transformed_center = Vector +  np.dot(r.as_matrix(), point)
    print(transformed_center)
    transformed_centers.append(transformed_center)
    
file_path = 'D:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\pent/transformed_centers_0.txt'
np.savetxt(file_path,transformed_centers)

[ 0.30941448  0.99358511 -0.10177808]
[-0.01174076  0.28923604  0.2646991 ]
[-0.3199594   0.37756237 -0.21296251]
[-0.35395736  1.06504214  0.06491204]
[ 0.84597683  2.4131182  -0.18225684]
[0.26599237 1.28261065 0.00149443]
[8.92898217e-02 1.53620613e+00 1.33446022e-03]
[-0.01864811  0.48540965 -0.04309069]
[-0.19345208  0.82854754 -0.05662258]
[ 0.46973996  2.21046813 -0.26461266]
[-0.24889663  1.26530564  0.35222954]
[-0.40548396  0.15632156 -0.1512071 ]
[ 0.17378107  0.98812467 -0.31040442]
[0.34418935 0.99905586 0.09545364]
[0.24155927 0.15135127 0.06591163]
[ 0.4962392   2.19090136 -0.48960495]
[0.24221557 0.04942653 0.13141996]
[ 0.31888482  0.61447698 -0.07476744]
[0.54931115 2.32288595 0.09755027]
[ 0.97019553  2.19918032 -0.0289808 ]
[-0.27382496  0.85039121 -0.23831992]
[ 0.05329093  1.33026779 -0.03645783]
[ 0.32604751  2.49041422 -0.37048445]
[-0.1574517   0.4432947  -0.15813245]
[-0.08970045  1.48557556 -0.37411535]
[ 1.01165125  2.20201524 -0.14862267]
[-0.32405859  1.13

In [99]:
for i in range(len(centers)):
    if not (np.array_equal(transformed_centers[i], centers[i])):
        print(transformed_centers[i] - centers[i])
        #print(Transformations[i])

[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.5399998 -0.16     ]
[ 0.6399996  0.53999

In [66]:
import open3d as o3d
import numpy as np
spheres = []

# Define the radius of the spheres
sphere_radius = 0.05  # Adjust the radius as needed
changed_sphere = o3d.geometry.TriangleMesh()
# Create a mesh sphere for each point and translate it to the point's location
count = 0
for i in range(transformed_centers.__len__()):
    point = transformed_centers[i]
    # Create a sphere mesh
    mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=sphere_radius)
    mesh_sphere.compute_vertex_normals()
    
    # Translate the sphere to the point's location
    mesh_sphere.translate(point)
    if not (np.array_equal(point, centers[i])):
        # Color the sphere red if its position is not in reference_centers
        mesh_sphere.paint_uniform_color([1.0, 0.0, 0.0])  # Red color
        changed_sphere += mesh_sphere
        count = count + 1 
    else:
        # Otherwise, color it light grey
        mesh_sphere.paint_uniform_color([0.7, 0.7, 0.7])  # Light grey color
    # Add the sphere to the list
    spheres.append(mesh_sphere)
print(spheres.__len__(), count)
# Combine all the spheres into one mesh
post_spheres = o3d.geometry.TriangleMesh()
for sphere in spheres:
    post_spheres += sphere

# Optionally, you can set the color of the spheres
#all_spheres.paint_uniform_color([0.7, 0.7, 0.7])  # Light grey color
mesh = o3d.io.read_triangle_mesh('D:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\output\pent/matrix_1/000000.obj')
mesh.compute_vertex_normals()
# Draw the combined mesh
o3d.visualization.draw_geometries([post_spheres, mesh])

1000 225


## Use transformed centers and centers to get indices and transformation dual quaternions

In [76]:
transformed_centers_path = 'D:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\pent/transformed_centers_0.txt'

loaded_transformed_centers = np.loadtxt(transformed_centers_path)
print(loaded_transformed_centers)

[[ 0.30941448  0.99358511 -0.10177808]
 [-0.01174076  0.28923604  0.2646991 ]
 [-0.3199594   0.37756237 -0.21296251]
 ...
 [ 0.67638053  2.10591121 -0.45961684]
 [ 0.00987653  0.13647208  0.1315317 ]
 [ 0.33245873  0.64176136 -0.19910005]]


In [108]:
def get_dual_quaternions(original_centers, transformed_centers):
    moved_indices = []
    for i in range(len(centers)):
        if not (np.array_equal(centers[i], transformed_centers[i])):
            moved_indices.append(i)
    dual_quaternions = np.zeros((len(centers), 8), dtype=np.float32)
    for i in moved_indices:
        original = centers[i]
        transformed = loaded_transformed_centers[i]
        # Compute rotation - assume for simplicity no rotation (identity quaternion)
        rotation_quaternion = R.from_quat([0, 0, 0, 1])  # Identity rotation
    
        # Compute translation as a vector
        translation = transformed - original
        
        # Convert rotation to a quaternion
        rotation_quat = rotation_quaternion.as_quat()
        
        # Dual quaternion: q + ϵ * (0.5 * t * q)
        translation_quat = np.hstack((translation, [0]))
        dual_quat = np.hstack((rotation_quat, 0.5 * translation_quat))
        dual_quaternions[i] = dual_quat
    return moved_indices, dual_quaternions

In [109]:
indices, dual_quaternions = get_dual_quaternions(centers, transformed_centers)
dual_quaternions

array([[ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       ...,
       [ 0.       ,  0.       ,  0.       , ...,  0.2699999, -0.08     ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ]], dtype=float32)

In [96]:
from scipy.spatial.transform import Rotation as R
transformed_centers = []
Transformations = []
for i in range(len(centers)):
    real_values = dual_quaternions[i][:4]  # Replace with actual Real values
    dual_values = dual_quaternions[i][4:]  # Replace with actual Dual values
    print(real_values, dual_values)
    


[0. 0. 0. 0.] [0. 0. 0. 0.]
[0. 0. 0. 0.] [0. 0. 0. 0.]
[0. 0. 0. 0.] [0. 0. 0. 0.]
[0. 0. 0. 0.] [0. 0. 0. 0.]
[0. 0. 0. 1.] [ 0.3199998  0.2699999 -0.08       0.       ]
[0. 0. 0. 0.] [0. 0. 0. 0.]
[0. 0. 0. 0.] [0. 0. 0. 0.]
[0. 0. 0. 0.] [0. 0. 0. 0.]
[0. 0. 0. 0.] [0. 0. 0. 0.]
[0. 0. 0. 1.] [ 0.3199998  0.2699999 -0.08       0.       ]
[0. 0. 0. 0.] [0. 0. 0. 0.]
[0. 0. 0. 0.] [0. 0. 0. 0.]
[0. 0. 0. 0.] [0. 0. 0. 0.]
[0. 0. 0. 0.] [0. 0. 0. 0.]
[0. 0. 0. 0.] [0. 0. 0. 0.]
[0. 0. 0. 1.] [ 0.3199998  0.2699999 -0.08       0.       ]
[0. 0. 0. 0.] [0. 0. 0. 0.]
[0. 0. 0. 0.] [0. 0. 0. 0.]
[0. 0. 0. 1.] [ 0.3199998  0.2699999 -0.08       0.       ]
[0. 0. 0. 1.] [ 0.3199998  0.2699999 -0.08       0.       ]
[0. 0. 0. 0.] [0. 0. 0. 0.]
[0. 0. 0. 0.] [0. 0. 0. 0.]
[0. 0. 0. 1.] [ 0.3199998  0.2699999 -0.08       0.       ]
[0. 0. 0. 0.] [0. 0. 0. 0.]
[0. 0. 0. 0.] [0. 0. 0. 0.]
[0. 0. 0. 1.] [ 0.3199998  0.2699999 -0.08       0.       ]
[0. 0. 0. 0.] [0. 0. 0. 0.]
[0. 0. 0. 0.] [0. 0.